## Problem 1

The liquid-vapor equilibrium equation with two parameters $A_{12}$ and $A_{21}$:

$
\begin{aligned}
P = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)P_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)P_{1,4 dioxane}^{sat}
\end{aligned}
$

The saturation pressures, $p^{sat}$ are given by the Antoine equation:

$
\log_{10}(P^{sat}) = a_1 - \frac{a_2}{T + a_3},
$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|     &nbsp;  | $a_1$   | $a_2$    | $a_3$   |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$, so $x_2=1-x_1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table:

### First step, formulating the least square problem.

Minimize the total error between expected result and experimental result:

$\underset{A_{12},A_{21}}{\text{min}}\sum^{11}_{i=1}(P(x_{1i},A_{12},A_{21})-P_i)^2$

Because the independent value is an exponent in an exponential function, the P function is nonlinear with respect to parameters $A_{12}$ and $A_{21}$.

Apply gradient descent:

In [5]:
%reset
import torch as t
from torch.autograd import Variable
import numpy as np

T = 20 # temperature in C
x1 = t.from_numpy(np.linspace(0,1,11))
x2 = 1 - x1
Pa = t.from_numpy(np.array([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5])) # experimental result
Psw = 10**(8.07131 - 1730.63/(T+233.426)) # for water
Pso = 10**(7.43155 - 1554.679/(T+240.337)) # for 1,4 dioxane

A = Variable(t.tensor([1.0, 2.0]), requires_grad=True)

a=0.001

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = t.norm((x1*t.exp(A[0]*((A[1]*x2)/(A[0]*x1+A[1]*x2))**2)*Psw + x2*t.exp(A[1]*((A[0]*x1)/(A[0]*x1+A[1]*x2))**2)*Pso - Pa)**2)
    loss.backward()
    print(A.data.numpy())
    print(loss.data.numpy())# no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        A -= a * A.grad

        # need to clear the gradient at every step, or otherwise it will accumulate...
        A.grad.zero_()

print(A.data.numpy())
print(loss.data.numpy())

[1. 2.]
66.91738389689313
[1.1842433 2.0323162]
37.00064220258844
[1.3149209 2.060182 ]
21.595400252024735
[1.409516  2.0808375]
13.621856361983486
[1.4756807 2.0905569]
9.99003600088483
[1.5172894 2.085962 ]
8.534503582427966
[1.5433757 2.0712738]
7.696276839601487
[1.5630636 2.0530381]
6.995869360187466
[1.5802917 2.0342982]
6.3627254976426295
[1.5963675 2.0160847]
5.786097800954398
[1.6117111 1.9987156]
5.261536942606003
[1.6264584 1.9822699]
4.7852003959977925
[1.6406534 1.9667451]
4.353376623843731
[1.6543117 1.9521111]
3.9624874283294345
[1.667441  1.9383284]
3.6091283034797774
[1.6800478 1.9253548]
3.290092641840061
[1.6921389 1.9131485]
3.002384208417624
[1.7037227 1.9016683]
2.743216638444469
[1.7148085 1.8908749]
2.51001768858285
[1.7254066 1.8807307]
2.300417625555958
[1.7355282 1.8712001]
2.1122406822322377
[1.745185 1.862249]
1.943492638936032
[1.7543893 1.8538457]
1.7923503890666497
[1.7631533 1.8459599]
1.657148461705761
[1.77149   1.8385631]
1.5363627999167364
[1.779412


2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1;
3. Compare your optimized model with the data. Does your model fit well with the data?

In [168]:
%reset
import torch as t
from torch.autograd import Variable
import numpy as np
import math

x1 = np.linspace(0,1,11)
x2 = 1 - x1
Pa = np.array([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]) # experimental result
Psw = 10**(8.07131 - 1730.63/(20+233.426)) # for water
Pso = 10**(7.43155 - 1554.679/(20+240.337)) # for 1,4 dioxane
A = [1.9542572, 1.6984679]
error = (x1*np.exp(A[0]*((A[1]*x2)/(A[0]*x1+A[1]*x2))**2)*Psw + x2*np.exp(A[1]*((A[0]*x1)/(A[0]*x1+A[1]*x2))**2)*Pso - Pa)**2
print(error**0.5)
print(np.mean(error**0.5))
print(np.sum(error))


[0.72409953 0.23471241 0.2434018  0.02183041 0.08681134 0.06561445
 0.08407494 0.02910153 0.11482018 0.10176937 0.02674792]
0.1575439876952868
0.6831441318591709
